In [26]:
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader
# https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html#data-handling-of-graphs

In [4]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [17]:
dataset

Cora()

In [6]:
len(dataset)

1

In [7]:
dataset.num_classes

7

In [8]:
dataset.num_node_features

1433

In [16]:
data = dataset[0]

In [14]:
data.is_undirected()

True

In [18]:
data

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [19]:
data.train_mask.sum().item()

140

In [21]:
data.val_mask.sum().item()

500

In [22]:
data.test_mask.sum().item()

1000

In [25]:
data.test_mask.sum().item()

1000

In [27]:
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [28]:
for batch in loader:
  batch

In [29]:
batch

Batch(batch=[2708], edge_index=[2, 10556], ptr=[2], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

TypeError: 'int' object is not callable

# GCN

In [36]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [37]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [40]:
model.eval()
_, pred = model(data).max(dim=1)
correct = int(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / int(data.test_mask.sum())
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.7960


# dataload

In [41]:
import numpy as np
import scipy.sparse as sp
import torch

In [42]:
def load_data(path="../data/cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

In [45]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [55]:
data = dataset[0]

In [56]:
data

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [58]:
x = data.x

In [64]:
x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [73]:
a =data.edge_index[0]

In [75]:
len(a)

10556

In [76]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2, 2, 3, 2, 4, 3, 4],
                           [1, 0, 2, 1, 3, 2, 4, 2, 4, 3]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1], [0], [-1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

In [77]:
data

Data(edge_index=[2, 10], x=[5, 1])

In [78]:
data.edge_index

tensor([[0, 1, 1, 2, 2, 3, 2, 4, 3, 4],
        [1, 0, 2, 1, 3, 2, 4, 2, 4, 3]])

In [3]:
x=3
def addx(a):
    return x+a

print(addx(4))

7
